## 合并ndvi和天气数据后处理为模型输入格式
## Merging NDVI and weather data and then processing them into a model input format

In [12]:
import xarray as xr
import numpy as np
import os

读取两个nc文件

In [2]:
ndvi_path = "ndvi_precombine.nc"
weather_path = "weather_precombine.nc"

ndvi_ds = xr.open_dataset(ndvi_path)
weather_ds = xr.open_dataset(weather_path)

ndvi_ds, weather_ds


(<xarray.Dataset> Size: 1MB
 Dimensions:  (x: 76, y: 49, time: 37)
 Coordinates:
   * x        (x) float64 608B 4.72e+05 4.72e+05 4.72e+05 ... 4.727e+05 4.727e+05
   * y        (y) float64 392B 5.48e+06 5.48e+06 5.48e+06 ... 5.48e+06 5.48e+06
   * time     (time) datetime64[ns] 296B 2025-05-07 2025-05-10 ... 2025-08-23
 Data variables:
     ndvi     (time, y, x) float64 1MB ...,
 <xarray.Dataset> Size: 4MB
 Dimensions:       (time: 37, y: 49, x: 76)
 Coordinates:
   * time          (time) datetime64[ns] 296B 2025-05-07 ... 2025-08-23
   * y             (y) float64 392B 5.48e+06 5.48e+06 ... 5.48e+06 5.48e+06
   * x             (x) float64 608B 4.72e+05 4.72e+05 ... 4.727e+05 4.727e+05
 Data variables:
     sm_30cm_mean  (time, y, x) float32 551kB ...
     RAIN_sum      (time, y, x) float32 551kB ...
     irrig_mm_sum  (time, y, x) float32 551kB ...
     IRRAD_sum     (time, y, x) float32 551kB ...
     TMIN_mean     (time, y, x) float32 551kB ...
     TMAX_mean     (time, y, x) float32

合并前做最后检查

last check

In [3]:
ndvi_ds = ndvi_ds.transpose("time", "y", "x")
weather_ds = weather_ds.transpose("time", "y", "x")

print("NDVI:", ndvi_ds.dims)
print("WEATHER:", weather_ds.dims)


NDVI: FrozenMappingWarningOnValuesAccess({'x': 76, 'y': 49, 'time': 37})
WEATHER: FrozenMappingWarningOnValuesAccess({'time': 37, 'y': 49, 'x': 76})


In [4]:
weather_aligned = weather_ds.sel(time=ndvi_ds.time, method="nearest")
weather_aligned = weather_aligned.assign_coords(time=ndvi_ds.time)

print("Aligned WEATHER time equals NDVI time:",
      np.array_equal(weather_aligned.time.values, ndvi_ds.time.values))


Aligned WEATHER time equals NDVI time: True


In [5]:
print("X equal:", np.array_equal(weather_aligned.x.values, ndvi_ds.x.values))
print("Y equal:", np.array_equal(weather_aligned.y.values, ndvi_ds.y.values))


X equal: True
Y equal: True


## 合并两个文件
## combine these 2 files

In [6]:
combined_ds = xr.merge([ndvi_ds, weather_aligned], compat="override")
combined_ds


<xarray.Dataset> Size: 6MB
Dimensions:       (x: 76, y: 49, time: 37)
Coordinates:
  * x             (x) float64 608B 4.72e+05 4.72e+05 ... 4.727e+05 4.727e+05
  * y             (y) float64 392B 5.48e+06 5.48e+06 ... 5.48e+06 5.48e+06
  * time          (time) datetime64[ns] 296B 2025-05-07 ... 2025-08-23
Data variables:
    ndvi          (time, y, x) float64 1MB ...
    sm_30cm_mean  (time, y, x) float32 551kB ...
    RAIN_sum      (time, y, x) float32 551kB ...
    irrig_mm_sum  (time, y, x) float32 551kB ...
    IRRAD_sum     (time, y, x) float32 551kB ...
    TMIN_mean     (time, y, x) float32 551kB ...
    TMAX_mean     (time, y, x) float32 551kB ...
    VAP_mean      (time, y, x) float32 551kB ...
    WIND_mean     (time, y, x) float32 551kB ...

保存   save

In [7]:
out_path = "combine.nc"
combined_ds.to_netcdf(out_path)
print("Saved:", out_path)


Saved: combine.nc


In [8]:
chk = xr.open_dataset("combine.nc")
chk


<xarray.Dataset> Size: 6MB
Dimensions:       (x: 76, y: 49, time: 37)
Coordinates:
  * x             (x) float64 608B 4.72e+05 4.72e+05 ... 4.727e+05 4.727e+05
  * y             (y) float64 392B 5.48e+06 5.48e+06 ... 5.48e+06 5.48e+06
  * time          (time) datetime64[ns] 296B 2025-05-07 ... 2025-08-23
Data variables:
    ndvi          (time, y, x) float64 1MB ...
    sm_30cm_mean  (time, y, x) float32 551kB ...
    RAIN_sum      (time, y, x) float32 551kB ...
    irrig_mm_sum  (time, y, x) float32 551kB ...
    IRRAD_sum     (time, y, x) float32 551kB ...
    TMIN_mean     (time, y, x) float32 551kB ...
    TMAX_mean     (time, y, x) float32 551kB ...
    VAP_mean      (time, y, x) float32 551kB ...
    WIND_mean     (time, y, x) float32 551kB ...

In [ ]:
ds = xr.open_dataset("combine.nc")
ds

for v in ds.data_vars:
    print(v, ds[v].dims, ds[v].shape)

ndvi ('time', 'y', 'x') (37, 49, 76)
sm_30cm_mean ('time', 'y', 'x') (37, 49, 76)
RAIN_sum ('time', 'y', 'x') (37, 49, 76)
irrig_mm_sum ('time', 'y', 'x') (37, 49, 76)
IRRAD_sum ('time', 'y', 'x') (37, 49, 76)
TMIN_mean ('time', 'y', 'x') (37, 49, 76)
TMAX_mean ('time', 'y', 'x') (37, 49, 76)
VAP_mean ('time', 'y', 'x') (37, 49, 76)
WIND_mean ('time', 'y', 'x') (37, 49, 76)


调整为模型输入格式

processee them into a model input format

In [14]:
out_dir = "arrays"
os.makedirs(out_dir, exist_ok=True)

for var in ds.data_vars:
    arr = ds[var].values   # numpy array, shape = (time, y, x)

    print(f"Saving {var}.npy, shape={arr.shape}, dtype={arr.dtype}")

    np.save(
        os.path.join(out_dir, f"{var}.npy"),
        arr.astype(np.float32)  # 和原仓库保持一致
    )


Saving ndvi.npy, shape=(37, 49, 76), dtype=float64
Saving sm_30cm_mean.npy, shape=(37, 49, 76), dtype=float32
Saving RAIN_sum.npy, shape=(37, 49, 76), dtype=float32
Saving irrig_mm_sum.npy, shape=(37, 49, 76), dtype=float32
Saving IRRAD_sum.npy, shape=(37, 49, 76), dtype=float32
Saving TMIN_mean.npy, shape=(37, 49, 76), dtype=float32
Saving TMAX_mean.npy, shape=(37, 49, 76), dtype=float32
Saving VAP_mean.npy, shape=(37, 49, 76), dtype=float32
Saving WIND_mean.npy, shape=(37, 49, 76), dtype=float32
